In [5]:
import rasterio
import numpy as np
import os
import shutil
import cv2
from glob import glob
from tqdm import tqdm

IMAGE = './dataset/train_img/*.tif'
MASK = './dataset/train_mask/*.tif'

OUTPUT_IMG = './dataset/images/'
OUTPUT_MASK = './dataset/annotations/'

if os.path.exists(OUTPUT_IMG): shutil.rmtree(OUTPUT_IMG)
if os.path.exists(OUTPUT_MASK): shutil.rmtree(OUTPUT_MASK)
os.mkdir(OUTPUT_IMG)
os.mkdir(OUTPUT_MASK)

MAX_PIXEL_VALUE = 65535 # Max. pixel value, used to normalize the image

def get_img_762bands(path):
    img = rasterio.open(path).read((7,6,2)).transpose((1, 2, 0))    
    img = np.float32(img)/MAX_PIXEL_VALUE
    
    return img

def get_mask_arr(path):
    img = rasterio.open(path).read().transpose((1, 2, 0))
    seg = np.float32(img)
    return seg

for img_path in tqdm(glob(IMAGE)):
    img = get_img_762bands(img_path)
    img = np.array(img * 255, dtype=np.uint8)
    cv2.imwrite(os.path.join(OUTPUT_IMG, img_path.split('/')[-1].replace('tif', 'png').replace('_img', '')), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

for mask_path in tqdm(glob(MASK)):
    mask = get_mask_arr(mask_path)
    mask = np.array(mask, dtype=np.uint8)
    cv2.imwrite(os.path.join(OUTPUT_MASK, mask_path.split('/')[-1].replace('tif', 'png').replace('_mask', '')), mask)

100%|██████████| 33575/33575 [08:27<00:00, 66.18it/s]
